In [1]:
import psycopg2
import pandas as pd
import json
import datetime

In [2]:
aujourd= datetime.datetime.today()
aujourd
print(f"{aujourd:%d %m  %Y}")

13 07  2022


In [2]:
def get_index(nom, nombre):
    data = {"a": [2, 1, 10], "b": [5,3,5] }
    l = data[nom]
    return l[nombre]

In [3]:
get_index(nom="b", nombre=1)

3

- Afficher pour chaque annexe, la liste des champs (./app/config.py)
- Pour chaque annexe, créer la requête pour récupérer les données concernant l'annexe de manière variabilisé
- Créer une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision. Cette fonction fait une requête à la base de donnée avec ces arguments. En retour elle renvoie un dataframe pandas 
- Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.
- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [29]:
import app.backend.config as conf
data = conf.CHAMPS_ANNEXES

In [ ]:
def annexe(nom):
    #l=conf.CHAMPS_ANNEXES
    return data[nom]
annexe(nom='DATA_EMPRUNT')

In [31]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")


def get_data(connexion,annexe, year):
    query= """SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = 'DATA_EMPRUNT'
    AND documentbudgetaire.exercice = '2020'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 100 """
    dat = pd.read_sql_query(query, conn)
    conn.close()
    return dat


In [32]:
get_data(connexion= conn, annexe= 'DATA_EMPRUNT', year='2020')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,57517,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""123461.87""}]",10063,21310584400152,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_ORGANI...",10063,SERVICE PORTAGE DE REPAS,31584,M4-M4,...,None,1,A,21310584400012,"[{""Nature"": ""61551"", ""ContNat"": ""011"", ""ArtSpe...",21310584400152,21310584400152,COMMUNE DE VILLEMUR SUR TARN,Commune,31
1,57593,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",10080,21170434100091,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",10080,BUDGET ANNEXE ASSAINISSEMENT,17434,M4-M49_D,...,None,1,A,21170434100018,"[{""Nature"": ""6063"", ""LibCpte"": ""Fournitures d'...",21170434100091,21170434100091,BUDGET ANNEXE ASSAINISSEMENT,Commune,17
2,57603,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",10082,21720316500030,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",10082,ASSAINISSEMENT DE SAINT REMY DES,72316,M4-M49_A,...,None,1,A,21720316500014,"[{""Nature"": ""658"", ""ContNat"": ""65"", ""ArtSpe"": ...",21720316500030,21720316500030,SAINT REMY DES MONTS,Commune,72
3,57624,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""7000.00""}]",10085,21850120300119,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",10085,Bâtiment-relais restaurant,85120,M4-M4,...,None,1,A,21850120300010,"[{""Nature"": ""6811"", ""ContNat"": ""042"", ""ArtSpe""...",21850120300119,21850120300119,LANDEVIEILLE,Commune,85
4,57686,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",10096,21280116100077,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",10096,20003 - SERVICE ASSAINISSEMENT,28116,M4-M49_A,...,None,1,P,None,"[{""Nature"": ""28156"", ""ContNat"": ""040"", ""ArtSpe...",21280116100077,21280116100077,COMMUNE DE COURVILLE SUR EURE,Commune,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,60261,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""129939.01""}]",10538,21390013700091,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",10538,ARBOIS PISCINE - 13002,39013,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""6135"", ""Fonction"": ""413"", ""ContNa...",21390013700091,21390013700091,VILLE D'ARBOIS,Commune,39
96,60291,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",10542,21330165800018,"[DATA_EMPRUNT, DATA_FORMATION, DATA_FISCALITE,...",10542,FARGUES SAINT-HILAIRE,33223165,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6042"", ""LibCpte"": ""Achats de pres...",21330165800018,21330165800018,COMMUNE FARGUES SAINT-HILAIRE,Commune,33
97,60300,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""22679.80""}]",10543,21360088500189,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",10543,Budget Annexe - Lotissements,36088,M14-M14_COM_500_3500,...,None,1,A,21360088500015,"[{""Nature"": ""3351"", ""ContNat"": ""040"", ""ArtSpe""...",21360088500189,21360088500189,ISSOUDUN,Commune,36
98,60335,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""38904.58""}]",10550,21250240500088,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",10550,BOIS Les Fins 065,25240,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""61524"", ""ContNat"": ""011"", ""ArtSpe...",21250240500088,21250240500088,Commune LES FINS,COMMUNE,25


In [55]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, numero_docbudge):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{numero_docbudge}' 
        LIMIT 10 """
    dat = pd.read_sql_query(query, conn)
    return dat
    

In [57]:
get_data(connexion=conn, annexe='DATA_APCP', year=2019, numero_docbudge= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,3297,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",554,20006983900047,"[DATA_PATRIMOINE, DATA_AMORTISSEMENT_METHODE, ...",554,DELEGATION PISCINES COMMUNAUTAIRES,None,M14-M14_COM_SUP3500,...,None,3,A,20006983900013,"[{""Nature"": ""2135"", ""Fonction"": ""4"", ""ContNat""...",20006983900047,20006983900047,CC COTE D'ALBATRE,Communauté de Communes dont la population est ...,None
1,3366,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",566,21200033500019,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",566,BASTIA BP,None,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""023"", ""Fonction"": ""01"", ""ArtSpe"":...",21200033500019,21200033500019,MAIRIE DE BASTIA,COMMUNE,20
2,4756,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",799,21030082800139,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",799,BUDGET TVA,03082,M14-M14_COM_SUP3500,...,None,3,A,21030082800014,"[{""Nature"": ""1641"", ""Fonction"": ""01"", ""ContNat...",21030082800139,21030082800139,VILLE DE COMMENTRY,Commune,03
3,4882,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",820,20006559700011,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_TIERS, DA...",820,Budget Principal,None,M57-M57,...,None,3,P,None,"[{""Nature"": ""6188"", ""Fonction"": ""633"", ""ContNa...",20006559700011,20006559700011,Communauté Urbaine Caen La Mer,ETABLISSEMENT PUBLIC,None
4,6657,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",1141,20007167800011,"[DATA_EMPRUNT, DATA_CHARGE, DATA_TIERS, DATA_C...",1141,AGGLOMERATION DU CHOLETAIS,None,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""023"", ""Fonction"": ""01"", ""ArtSpe"":...",20007167800011,20007167800011,AGGLOMERATION DU CHOLETAIS,EPCI,49
5,7243,DATA_APCP,"[{""CodTypAutori"": ""E"", ""CodSTypAutori"": ""R"", ""...",1237,24330031600201,"[DATA_APCP, DATA_SIGNATURE, DATA_SIGNATAIRE, D...",1237,84-Z.A.C des quais de Floirac,None,M57-M57,...,None,1,A,24330031600011,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",24330031600201,24330031600201,Bordeaux Métropole,Etat,None
6,7774,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",1329,20003607700051,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",1329,EAU POTABLE SERV. AFFERME. DSP,,M4-M49_D,...,None,1,A,20003607700010,"[{""Nature"": ""605"", ""LibCpte"": ""Achats d eau"", ...",20003607700051,20003607700051,COM.COM DU GOLFE DE SAINT TROPEZ,,
7,10360,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",1798,21760351301860,"[DATA_PROVISION, DATA_APCP, DATA_SIGNATURE, DA...",1798,PARCS DE STATIONNEMENT,None,M4-M4,...,None,1,A,21760351300011,"[{""Nature"": ""1313"", ""ContOp"": ""2"", ""CodRD"": ""R...",21760351301860,21760351301860,VILLE DU HAVRE,COMMUNE,None
8,11781,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",2067,21370261600250,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",2067,BUDGET CIMETIERES VILLE DE TOURS,37261,M4-M4,...,None,1,A,21370261600011,"[{""Nature"": ""002"", ""ArtSpe"": ""false"", ""CodRD"":...",21370261600250,21370261600250,Ville de Tours,COMMUNE,37
9,24718,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",4365,22760540902270,"[DATA_CONSOLIDATION, DATA_PATRIMOINE, DATA_APC...",4365,Parc Départemental,76,M52-M52,...,None,1,A,22760540900019,"[{""Nature"": ""281351"", ""ContNat"": ""040"", ""ArtSp...",22760540902270,22760540902270,Département de Seine-Maritime,Départements,None


In [3]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire 
     ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite 
     ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 LIMIT 100
 """
dat = pd.read_sql_query(query, conn)
conn.close()



/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [6]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

In [7]:
df_min

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21630291900016,Budget Communal M14,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21630291900016,Commune de PUY-GUILLAUME,Commune
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""09"", ""CodProfilAmort"": ""P"", ""...",21750001600019,BUDGET PRINCIPAL,M57-M57,2020,Compte administratif,2,Principal,21750001600019,Ville de Paris,Commune
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""51265.65""}]",21630463400043,BUDGET EAU,M4-M49_D,2020,Compte administratif,1,Annexe,21630463400043,Commune de Viscomtat,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""50493.40""}]",26390475700014,Budget CCAS 42000,M14-M14_CCAS_INF3500,2020,Compte administratif,1,Principal,26390475700014,Commune de SAINT AMOUR,Commune
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21330331600011,74000 BUDGET COMMUNAL M14,M14-M14_COM_INF500,2020,Compte administratif,1,Principal,21330331600011,MAIRIE DE PONDAURAT,Commune


In [12]:
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    #print(json_df)
    #json_df.assign(**row)

In [15]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('./csv/2020-CA-Limit100.csv', index=False)  

In [9]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']